In [1]:
import warnings
# warnings. simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np
from tabulate import tabulate


In [123]:
data = GoldenCrossverSignal('RELIANCE')
data.to_csv('data.csv')
len(data)

9

In [99]:
required_df = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]

In [100]:
data[data['Position'] == 'Sell'].index[-1]

Timestamp('2023-08-01 00:00:00')

In [2]:

# Name, Entry TIme, Entry PRice, QTY, Exit Time, Exit Price

class Backtest:
    def __init__(self):
        self.columns = ['Equity Name', 'Trade', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'Quantity', 'Position Size', 'PNL', '% PNL']
        self.backtesting = pd.DataFrame(columns=self.columns)

    def buy(self, equity_name, entry_time, entry_price, qty):
        self.trade_log = dict(zip(self.columns, [None] * len(self.columns)))
        self.trade_log['Trade'] = 'Long Open'
        self.trade_log['Quantity'] = qty
        self.trade_log['Position Size'] = round(self.trade_log['Quantity'] * entry_price, 3)
        self.trade_log['Equity Name'] = equity_name
        self.trade_log['Entry Time'] = entry_time
        self.trade_log['Entry Price'] = round(entry_price, 2)

    def sell(self, exit_time, exit_price, exit_type, charge):
        self.trade_log['Trade'] = 'Long Closed'
        self.trade_log['Exit Time'] = exit_time
        self.trade_log['Exit Price'] = round(exit_price, 2)
        self.trade_log['Exit Type'] = exit_type
        self.trade_log['PNL'] = round((self.trade_log['Exit Price'] - self.trade_log['Entry Price']) * self.trade_log['Quantity'] - charge, 3)
        self.trade_log['% PNL'] = round((self.trade_log['PNL'] / self.trade_log['Position Size']) * 100, 3)
        self.trade_log['Holding Period'] = exit_time - self.trade_log['Entry Time']
        # self.backtesting = self.backtesting.append(self.trade_log, ignore_index=True)
        self.backtesting.loc[len(self.backtesting)] = self.trade_log
    def stats(self):
        df = self.backtesting
        parameters = ['Total Trade Scripts', 'Total Trade', 'PNL',  'Winners', 'Losers', 'Win Ratio','Total Profit', 'Total Loss', 'Average Loss per Trade', 'Average Profit per Trade', 'Average PNL Per Trade', 'Risk Reward']
        total_traded_scripts = len(df['Equity Name'].unique())
        total_trade = len(df.index)
        pnl = df.PNL.sum()
        winners = len(df[df.PNL > 0])
        loosers = len(df[df.PNL <= 0])
        win_ratio = str(round((winners/total_trade) * 100, 2)) + '%'
        total_profit = round(df[df.PNL > 0].PNL.sum(), 2)
        total_loss  = round(df[df.PNL <= 0].PNL.sum(), 2)
        average_loss_per_trade = round(total_loss/loosers, 2)
        average_profit_per_trade = round(total_profit/winners, 2)
        average_pnl_per_trade = round(pnl/total_trade, 2)
        risk_reward = f'1:{-1 * round(average_profit_per_trade/average_loss_per_trade, 2)}'
        data_points = [total_traded_scripts,total_trade,pnl,winners, loosers, win_ratio, total_profit, total_loss, average_loss_per_trade, average_profit_per_trade, average_pnl_per_trade, risk_reward]
        data = list(zip(parameters,data_points ))
        print(tabulate(data, ['Parameters', 'Values'], tablefmt='psql'))
        

In [23]:
from datetime import datetime, timedelta
seven_days_ago = datetime.now() - timedelta(days=30)
today_date = datetime.now()- timedelta(days=7)
stocks_to_buy =[]
def GoldenCrossverSignal(name,x,y):
    path = f'./Data/{name}.csv'
    data = pd.read_csv(path, parse_dates=['Date'], index_col='Date') #index_col='Date'
    # data['Date'] = pd.to_datetime(data['Date'])
    
    data['Prev_Close'] = data.Close.shift(1)
    data['20_SMA'] = data.Prev_Close.rolling(window=20, min_periods=1).mean()
    data['50_SMA'] = data.Prev_Close.rolling(window=50, min_periods=1).mean()
    data['20_SMA_diff'] =  data['20_SMA'] - data['20_SMA'].shift(7)
    data['50_SMA_diff'] = data['50_SMA'].shift(-14) - data['50_SMA']
    data = data.iloc[-x:y]
    data['Signal'] = 0
    data['Signal'] = np.where((data['50_SMA_diff'] > 0), 1, 0)
    data['Position'] = data.Signal.diff()
    data['Position'] = data["Position"].shift(14)
    df_pos = data[(data['Position'] == 1) | (data['Position'] == -1)].copy()
    # df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x == 1 else 'Sell')
    # required_df = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]
    
    
    # df_pos["newPosition"]=df_pos["Position"].shift(14) #new add
    # df_pos.to_csv("test33.csv")
    # print(df_pos.columns)
    # Filter the DataFrame based on the conditions
    # try:
    #     filtered_df = df_pos.loc[(df_pos["Date"] >= seven_days_ago)]
    #     if len(filtered_df) : 
    #         # print(filtered_df['Position'].iat[-1] )
    #         if   filtered_df['Position'].iat[-1] == "Buy":
    #             stocks_to_buy.append(name)
    #             sig_date = filtered_df['Date'].iloc[0]
    #             z= sig_date - pd.Timedelta(days=0)
    #             old_price = filtered_df.loc[filtered_df['Date'] == z]['Close'].iloc[0]
    #             cur_price = data.loc[data['Date'] >= today_date]['Close'].iloc[0]
    #             perc = ((cur_price-old_price)/old_price)*100
    #             print(name,perc,sig_date)
    # except Exception as e:
    #     print(f"error {name} , {e}")
    

    return  df_pos
sty='3PLAND'
GoldenCrossverSignal(sty,0,-1)

,Open,High,Low,Close,Adj Close,Volume,Prev_Close,20_SMA,50_SMA,20_SMA_diff,50_SMA_diff,Signal,Position
Date,,,,,,,,,,,,,
2014-03-21,7.050000,7.050000,7.050000,7.050000,7.050000,0,7.050000,7.0475,7.098864,0.0750,0.049136,1,Buy
2014-04-02,7.750000,7.750000,7.750000,7.750000,7.750000,1,8.000000,7.2200,7.106000,0.1650,0.176000,1,Sell
2014-04-03,7.750000,7.750000,7.750000,7.750000,7.750000,0,7.750000,7.2475,7.109000,0.1850,0.215000,1,Buy
2014-09-24,11.650000,11.700000,11.350000,11.700000,11.700000,4900,11.150000,12.4125,13.185000,-0.3850,-0.894000,0,Sell
2015-03-02,10.600000,10.600000,10.600000,10.600000,10.600000,70000,11.000000,10.2175,10.243000,0.0400,0.590000,1,Buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-27,23.150000,24.500000,22.900000,23.850000,23.850000,10435,23.500000,22.6450,21.839000,1.2475,0.404000,1,Buy
2023-06-26,19.299999,20.000000,19.150000,19.549999,19.549999,2651,19.299999,21.5050,22.489000,-0.5475,-1.161000,0,Sell
2023-09-07,22.900000,23.100000,22.299999,23.000000,23.000000,35633,22.250000,20.7575,20.104000,0.6250,0.871000,1,Buy


In [5]:
# required_df = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]

In [24]:
bt = Backtest()
capital = 500000
stocks = pd.read_csv('.\ind_nifty100list.csv')
scripts = stocks["Symbol"]
# scripts=["IEX"]
for stock in scripts:
    data = GoldenCrossverSignal(stock,0,-1)
    try :
        required_df = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]
        
        for index, data in required_df.iterrows():
            if(data.Position == 'Buy'):
                qty = capital // data.Open
                bt.buy(stock, index, data.Open, qty)
            else:
                bt.sell(index, data.Open, 'Exit Trigger', 0)
    except :
        pass

In [25]:
bt.backtesting

,Equity Name,Trade,Entry Time,Entry Price,Exit Time,Exit Price,Quantity,Position Size,PNL,% PNL
0,ABB,Long Closed,2014-02-25,680.2,2015-03-31,1220.1,735.0,499947.009,396826.5,79.374
1,ABB,Long Closed,2015-04-10,1355.0,2015-05-13,1327.8,369.0,499995.000,-10036.8,-2.007
2,ABB,Long Closed,2015-06-03,1302.1,2015-06-29,1330.0,383.0,498704.291,10685.7,2.143
3,ABB,Long Closed,2015-07-10,1380.5,2015-08-28,1212.0,362.0,499741.000,-60997.0,-12.206
4,ABB,Long Closed,2015-12-01,1172.6,2015-12-02,1175.0,426.0,499527.590,1022.4,0.205
...,...,...,...,...,...,...,...,...,...,...
1911,ZYDUSLIFE,Long Closed,2020-03-24,297.0,2021-02-26,434.2,1683.0,499851.000,230907.6,46.195
1912,ZYDUSLIFE,Long Closed,2021-04-20,542.0,2021-08-04,590.0,922.0,499724.000,44256.0,8.856
1913,ZYDUSLIFE,Long Closed,2022-06-01,371.0,2022-06-17,342.5,1347.0,499737.000,-38389.5,-7.682
1914,ZYDUSLIFE,Long Closed,2022-07-12,370.0,2022-08-10,362.0,1351.0,499870.000,-10808.0,-2.162


In [15]:
# bt.backtesting.to_csv('RELIANCE_TEST.csv')

In [26]:
bt.stats()

+--------------------------+--------------+
| Parameters               | Values       |
|--------------------------+--------------|
| Total Trade Scripts      | 101          |
| Total Trade              | 1916         |
| PNL                      | 72391692.38  |
| Winners                  | 873          |
| Losers                   | 1043         |
| Win Ratio                | 45.56%       |
| Total Profit             | 108926048.72 |
| Total Loss               | -36534356.34 |
| Average Loss per Trade   | -35028.15    |
| Average Profit per Trade | 124772.11    |
| Average PNL Per Trade    | 37782.72     |
| Risk Reward              | 1:3.56       |
+--------------------------+--------------+


In [201]:
required_df.to_csv("reliance_req.csv")

PermissionError: [Errno 13] Permission denied: 'reliance_req.csv'

In [ ]:
data.head()

Open         615.950012
High         625.549988
Low          615.049988
Close             620.5
Adj Close         620.5
Name: 2023-09-04 00:00:00, dtype: object